# December 03, 2023

https://adventofcode.com/2023/day/3

In [6]:
import re
#from collections import defaultdict

In [2]:
fn = "../data/2023/03.txt"
with open(fn, "r") as file:
    text = file.readlines()

text = [x[:-1] for x in text]

In [139]:
class Schematic:
    def __init__( self, text ):
        '''text as list of strings'''

        self.nrow = len(text)
        self.ncol = len(text[0])
        self.parts = list()
        self.symbols = list()

        for line in text:
            line_parts, line_symbols = self.__parse_line__( line )
            self.parts.append( line_parts )
            self.symbols.append( line_symbols )    

    ### INIT METHODS ###
    def __parse_line__( self, line ):
        '''return a dict of parts {pos: num} and symbols {pos: symbol}'''

        buffer = line
        parts = {}
        symbols = {}
        # read all the part ids and record their line positions

        cursor = 0
        while cursor < len(line):
            m = re.match( "(\.*)(\d+|[^\d\.]|$)", line[cursor:])

            if len(m[2]) == 0:
                break

            
            if re.match("\d", m[2]):
                # record number and position of part number
                parts[cursor + len(m[1])] = int(m[2])
            else:
                # record symbol and position
                symbols[cursor + len(m[1])] = m[2]

            # advance cursor
            cursor += len(m[1]) + len(m[2])
        
        return parts, symbols
    
    ### PART 1 METHODS ###
    def __check_for_symbol__( self, row, col ):
        if (row > 0 and row < self.nrow and
            col > 0 and col < self.ncol and 
            col in self.symbols[row].keys()):
                return self.symbols[row][col]
        
        return None

    def __find_part_symbol__( self, part_id, row, col ):
        '''return the first symbol found or None if no symbols adjacent'''

        # rightmost position of the part_id
        rcol = col + len(str(part_id)) - 1

        # check above
        for pos in range( col-1, rcol+2 ):
            sym = self.__check_for_symbol__( row-1, pos )
            if sym is not None:
                return sym
    
        # check left/right
        sym = self.__check_for_symbol__( row, col-1 )
        if sym is not None:
            return sym
        sym = self.__check_for_symbol__( row, rcol+1 )
        if sym is not None:
            return sym
        
        # look out below!
        for pos in range( col-1, rcol+2 ):
            sym = self.__check_for_symbol__( row+1, pos )
            if sym is not None:
                return sym

        return None
    
    def sum_part_numbers( self ):
        '''sum part numbers that are adjacent to a symbold'''
        tot = 0
        for i, d in enumerate(self.parts):
            for j, id in d.items():
                if self.__find_part_symbol__( id, i, j ) is not None:
                    tot += id
                #print(i, j, id, tot)
        return tot
    
    ### PART 2 METHODS ###
    @staticmethod
    def __part_is_adjacent__( id, part_row, part_col, sym_row, sym_col ):
        part_rcol = part_col + len(str(id)) - 1

        # check above/on/below row
        if abs(part_row - sym_row) > 1:
            return False

        # check col
        if (sym_col >= part_col - 1) and (sym_col <= part_rcol + 1):
            return True

        return False
    
    def __find_adjacent_parts__( self, sym_row, sym_col ):
        adj_parts = []
        # find adj parts on row above
        if sym_row > 0:
            for col, id in self.parts[sym_row-1].items():
                if self.__part_is_adjacent__( id, sym_row-1, col, sym_row, sym_col ):
                    adj_parts.append( id )

        # find adj parts on same row
        for col, id in self.parts[sym_row].items():
                if self.__part_is_adjacent__( id, sym_row-1, col, sym_row, sym_col ):
                    adj_parts.append( id )

        if sym_row < self.nrow - 1:
            for col, id in self.parts[sym_row+1].items():
                if self.__part_is_adjacent__( id, sym_row+1, col, sym_row, sym_col ):
                    adj_parts.append( id )

        return adj_parts
    
    def __gear_ratio__( self, sym, sym_row, sym_col ):
        '''returns gear_ratio for gears and 0 for non-gears'''
        if sym != "*":
            return 0
        
        adj_parts = self.__find_adjacent_parts__( sym_row, sym_col )
        
        #print("* at", sym_row, sym_col)
        #print(adj_parts)
        if len( adj_parts ) != 2:
            return 0
        
        #print("Found gear at", sym_row, sym_col )
        #print("Parts", *adj_parts )
        return adj_parts[0] * adj_parts[1]
    
    def sum_gear_ratios( self ):
        gear_tot = 0
        for sym_row, sym_dict in enumerate(self.symbols):
            for sym_col, sym in sym_dict.items():
                gear_tot += self.__gear_ratio__( sym, sym_row, sym_col )
        
        return gear_tot

    




         

        

### Dev

In [47]:
test = [
    '467..114..',
    '...*......',
    '..35..633.',
    '......#...',
    '617*......',
    '.....+.58.',
    '..592.....',
    '......755.',
    '...$.*....',
    '.664.598..',
]


In [137]:
schem = Schematic(test)
schem.sum_part_numbers()


4361

In [138]:
schem.sum_gear_ratios()

* at 1 3
[467, 35]
Found gear at 1 3
Parts 467 35
* at 4 3
[617]
* at 8 5
[755, 598]
Found gear at 8 5
Parts 755 598


467835

### Part 1

In [140]:
p1 = Schematic(text)
p1.sum_part_numbers()

557705

### Part 2

In [141]:
p2 = Schematic(text)
p2.sum_gear_ratios()

84266818